# ECODE example

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron
import plotting
import MEAutility as mu

import sys
import shutil

from pathlib import Path
import os

%matplotlib notebook

In [ ]:
sys.path.append('../ecode/')

In [ ]:
from generate_ecode_rheobase import generate_ecode_protocols, compute_rheobase_for_model, run_ecode_protocols, \
    interpolate_response, save_intracellular_responses, save_extracellular_template
                                    

In [ ]:
probe_type = "planar" # linear 
model_name = 'hay' # "hallerman"
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# output folder for data
output_folder = f"../data/{model_name}_ecode_probe_{probe_type}"

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe = None

param_names = [param.name for param in cell.params.values() if not param.frozen]
if model_name == "hallermann":
    cvode_active = False
else:
    cvode_active = True
sim = ephys.simulators.LFPySimulator(cell, cvode_active=cvode_active, electrode=probe)

In [ ]:
rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, step_min=0.34)

In [ ]:
ecode_protocols = generate_ecode_protocols(rheobase_current=rheobase, record_extra=True,
                                           protocols_with_lfp="firepattern")

In [ ]:
responses_dict = run_ecode_protocols(protocols=ecode_protocols, cell=cell, sim=sim, 
                                     resample_rate_khz=40)

In [ ]:
save_intracellular_responses(responses_dict=responses_dict, output_folder=output_folder)

In [ ]:
eap, locations = save_extracellular_template(responses_dict["firepattern"], ecode_protocols, "firepattern",
                                             probe, output_folder, response_id=1, fs=20, fcut=[300, 6000])

In [ ]:
ax = mu.plot_mea_recording(eap, probe)